# Simulating IPFS and IPNS Systems

This notebook provides a Python-based simulation of IPFS (InterPlanetary File System) and IPNS (InterPlanetary Naming System) to test various linking strategies for storing and retrieving IPAROs.

The notebook uses three classes to simulate these systems:
- **IPARO**: Represents the storage object on IPFS.
- **IPNS**: Keeps track of the latest capture for different websites.
- **IPFS**: Simulates the hashing, storage, and retrieval of IPARO objects.

The goal of the simulation is to test various linking strategies.

In [2]:
# Importing the necessary libraries
import hashlib
import random

## IPARO Object

**Properties:**
- `CID`: The CID (Content Identifier) generated by IPFS.
- `Data`: The data of the capture.
- `Linked Node CID(s)`: The CID(s) of the nodes linked to it.

**Functions:**
- `get_cid`: Returns the CID of the IPARO.
- `get_linked_cids`: Returns the CID(s) of the linked node(s).
- `get_content`: Returns the content of the IPARO.
- `__str__`: Returns a string representation of the IPARO object.

In [3]:
class IPARO:
    def __init__(self, cid: str, linked_cids: list, content: str, timestamp: str):
        """
        Initialize an IPARO object with its CID, linked CID(s), and content.

        Args:
            cid (str): The CID of the IPARO.
            linked_cids (list): List of CIDs of linked nodes.
            content (str): The content of the IPARO.
        """
        self.__cid = cid
        self.__linked_cids = linked_cids
        self.__content = content
        self.__timestamp = timestamp

    def get_cid(self) -> str:
        '''
        Returns the CID of the IPARO.

        Returns:
            str: The CID of the IPARO.
        '''
        return self.__cid

    def get_linked_cids(self) -> list:
        '''
        Returns the CID(s) of linked nodes.

        Returns:
            list: List of linked node CIDs.
        '''
        return self.__linked_cids

    def get_content(self) -> str:
        '''
        Returns the content of the IPARO.

        Returns:
            str: The content stored in the IPARO.
        '''
        return self.__content

    def get_timestamp(self) -> str:
        '''
        Returns the timestamp of the IPARO

        Returns:
            str: The timestamp in seconds since the epoch
        '''
        return self.__timestamp

    def __str__(self):
        '''
        Returns a string representation of the IPARO object.

        Returns:
            str: A string containing the CID, linked CID(s), and content of the IPARO.
        '''
        iparo = {
            "CID": self.__cid,
            "Content": self.__content,
            "Linked CID(s)": self.__linked_cids,
            "Timestamp": self.__timestamp,
        }
        return str(iparo)

## IPNS Object

**Description:**
- The IPNS class stores and maps the latest CID of a website.
- Tracks the number of operations (get and update) performed.

**Functions:**
- `update`: Updates the latest CID of a website.
- `get_cid`: Retrieves the CID of the latest capture for a website.
- `get_counts`: Returns the number of operations performed.
- `reset_counts`: Resets the counters for operations.

In [4]:
class IPNS:
    def __init__(self):
        """
        Initialize the IPNS object with an empty hashmap for storing CIDs 
        and counters for tracking operations.
        """
        self.data = {}
        self.update_count = 0
        self.get_count = 0

    def update(self, url, cid):
        '''
        Updates the latest CID for a given URL.

        Args:
            url (str): The URL of the website.
            cid (str): The CID of the latest capture.
        '''
        self.update_count += 1
        self.data[url] = cid

    def get_cid(self, url) -> str:
        '''
        Retrieves the latest CID for a given URL.

        Args:
            url (str): The URL of the website.

        Returns:
            str: The CID of the latest capture for the given URL.
        '''
        self.get_count += 1
        return self.data[url]

    def get_counts(self) -> dict:
        '''
        Returns the number of update and get operations performed.

        Returns:
            dict: Dictionary with the counts of update and get operations.
        '''
        counts = {"get": self.get_count, "update": self.update_count}
        return counts

    def reset_counts(self):
        """
        Resets the operation counters.
        """
        self.update_count = 0
        self.get_count = 0

## IPFS Object

**Description:**
- The IPFS class stores the nodes and simulates the hashing, storage, and retrieval operations.
- Tracks the number of operations (hash, store, retrieve).

**Functions:**
- `hash`: Hashes the content of a node to generate its CID.
- `store`: Stores a node with its CID.
- `retrieve`: Retrieves a node using its CID.
- `get_counts`: Returns the number of operations performed.
- `reset_counts`: Resets the counters for operations.

In [5]:
class IPFS:
    def __init__(self):
        '''
        Initialize the IPFS object with an empty hashmap for storing nodes
        and counters for tracking operations.
        '''
        self.data = {}
        self.hash_count = 0
        self.store_count = 0
        self.retrieve_count = 0

    def hash(self, content: str) -> str:
        '''
        Hashes the content to generate a CID.

        Args:
            content (str): The content of the node.

        Returns:
            str: The generated CID.
        '''
        sha256_hash = hashlib.sha256(content.encode()).hexdigest()
        self.hash_count += 1
        return 'Qm' + sha256_hash[:34]

    def store(self, cid: str, node: IPARO):
        '''
        Stores a node with its CID.

        Args:
            cid (str): The CID of the node.
            node (IPARO): The IPARO object to store.
        '''
        self.store_count += 1
        self.data[cid] = node

    def retrieve(self, cid) -> IPARO:
        '''
        Retrieves a node using its CID.

        Args:
            cid (str): The CID of the node to retrieve.

        Returns:
            IPARO: The retrieved IPARO object.
        '''
        self.retrieve_count += 1
        return self.data[cid]

    def get_counts(self) -> dict:
        '''
        Returns the number of hash, store, and retrieve operations performed.

        Returns:
            dict: Dictionary with counts of hash, store, and retrieve operations.
        '''
        counts = {"hash": self.hash_count, "store": self.store_count,
                  "retrieve": self.retrieve_count}
        return counts

    def reset_counts(self):
        """
        Resets the operation counters.
        """
        self.hash_count = 0
        self.store_count = 0
        self.retrieve_count = 0

    def reset_data(self):
        self.data = {}

    def get_data(self) -> dict:
        """Returns the data stored by IPFS (for debugging)."""
        return self.data

## Initialization and Operation Tracking

Here, we initialize the IPFS and IPNS objects and define a helper function `get_op_counts()` to display the number of operations performed.


In [6]:
# Initializing the simulated IPFS and IPNS
ipfs = IPFS()
ipns = IPNS()


def get_op_counts():
    '''
    Displays the number of operations performed by IPNS and IPFS.
    '''
    print("Number of operations IPNS performed:")
    print(ipns.get_counts())
    print("Number of operations IPFS performed:")
    print(ipfs.get_counts())

## Testing Different Linking Strategies

### 1. Linking to Only the Previous Node

In this test, each node will link only to the previous node in the chain. This strategy will be used to simulate a simple sequential storage system.


#### Storing Nodes

In [7]:
import time

# Testing parameters
NODE_NUM = 100
URL = "example.com"

# Create and store the first node
content = "Node 0"
timestamp = time.time()
to_be_hashed = str({
    "content": content,
    "timestamp": timestamp
})
cid = ipfs.hash(to_be_hashed)
first_node = IPARO(cid=cid, linked_cids=[],
                   content=content, timestamp=timestamp)
ipfs.store(cid, first_node)
ipns.update(URL, cid)


# Automate the creation of additional nodes
for i in range(1, NODE_NUM):
    content = f"Node {i}"
    timestamp = time.time()
    to_be_hashed = str({
        "content": content,
        "timestamp": timestamp
    })
    cid = ipfs.hash(to_be_hashed)
    linked_cids = [ipns.get_cid(URL)]  # Link to the previous node
    node = IPARO(cid=cid, linked_cids=linked_cids,
                 content=content, timestamp=timestamp)
    ipfs.store(cid, node)
    ipns.update(URL, cid)

get_op_counts()  # Output operation counts

Number of operations IPNS performed:
{'get': 99, 'update': 100}
Number of operations IPFS performed:
{'hash': 100, 'store': 100, 'retrieve': 0}


#### Retrieving Nodes

The following section tests retrieval of nodes by simulating a random node search.


In [8]:
# Reset the operation counts
ipfs.reset_counts()
ipns.reset_counts()

# Pick a random node to search for
node_num = random.randint(0, NODE_NUM - 1)
target_content = f"Node {node_num}"
print(f"Looking for node with content: {target_content}")

# Traverse back through the linked nodes to find the target
latest_node_cid = ipns.get_cid(URL)
node = ipfs.retrieve(latest_node_cid)
while node.get_content() != target_content:
    node = ipfs.retrieve(node.get_linked_cids()[0])

# Output the found node
print(f"Found node: {node}")
get_op_counts()

Looking for node with content: Node 20
Found node: {'CID': 'Qm9a93392d9a0d6603ac063e218607fe1d2b', 'Content': 'Node 20', 'Linked CID(s)': ['Qmd3193f108568a0e4f814dd45f933506522'], 'Timestamp': 1736881214.984975}
Number of operations IPNS performed:
{'get': 1, 'update': 0}
Number of operations IPFS performed:
{'hash': 0, 'store': 0, 'retrieve': 80}


### 2. Linking to all previous nodes

In this test, each node will link to all the previous nodes in the chain.

#### Storing Nodes

In [9]:
# Resetting IPFS from the last test
ipfs.reset_data()

# Testing parameters
NODE_NUM = 100
URL = "example.com"

# Create and store the first node
content = "Node 0"
timestamp = time.time()
to_be_hashed = str({
    "content": content,
    "timestamp": timestamp
})
cid = ipfs.hash(to_be_hashed)
first_node = IPARO(cid=cid, linked_cids=[],
                   content=content, timestamp=timestamp)
ipfs.store(cid, first_node)
ipns.update(URL, cid)

# Create and store the second node
content = "Node 1"
timestamp = time.time()
to_be_hashed = str({
    "content": content,
    "timestamp": timestamp
})
cid = ipfs.hash(to_be_hashed)
linked_cids = [ipns.get_cid(URL)]
second_node = IPARO(cid=cid, linked_cids=linked_cids,
                    content=content, timestamp=timestamp)
ipfs.store(cid, second_node)
ipns.update(URL, cid)

From here, there are 2 ways of creating a new node for this linking strategy, since this is a simulation, no data corruption can happen but that might not be true in practice. When retrieving the latest node which should contain the CIDs of all the previous node, two scenarios can happen:
1. The data is intact and the CIDs in the list is "correct" (which we really can't know for sure) and we can just add it to the new node we're creating
2. The data is corrupt and one or more of the CIDs is wrong or unfinished, in which case we have to recheck every CID to rebuild a new list of linked CIDs (not to mention fixing all the corrupted nodes)

So, for the purpose of this simulation, we will perform a check for every CID in the linked CID list of an IPARO to simulate the worst case scenario every time

In [10]:
# To automate adding the rest of the nodes
for i in range(2, NODE_NUM):
    content = "Node "+str(i)
    timestamp = time.time()
    to_be_hashed = str({
        "content": content,
        "timestamp": timestamp
    })
    cid = ipfs.hash(to_be_hashed)
    latest_node_cid = ipns.get_cid(URL)
    latest_node = ipfs.retrieve(latest_node_cid)
    linked_cids = []
    latest_node_linked_cids = latest_node.get_linked_cids()
    linked_cids = latest_node_linked_cids
    for link_cid in latest_node_linked_cids:
        ipfs.retrieve(link_cid)
        # Checking and repairing nodes goes here
    linked_cids.append(latest_node_cid)
    node = IPARO(cid=cid, linked_cids=linked_cids,
                 content=content, timestamp=timestamp)
    ipfs.store(cid, node)
    ipns.update(URL, cid)

# print(ipfs.get_data())
get_op_counts()  # Output operation counts

Number of operations IPNS performed:
{'get': 100, 'update': 100}
Number of operations IPFS performed:
{'hash': 100, 'store': 100, 'retrieve': 5029}


In this worst case scenario, where we have to retrieve and verify every CIDs in the linked CIDs of an IPARO, the retrieve count goes to almost 5000 (if we're storing 100 nodes)\
Of course this trade off makes it really easy to navigate to all the nodes just from the latest nodes

#### Retrieving nodes

The following section tests retrieval of nodes by simulating a random node search.

In [11]:
# Reset the operation counts
ipfs.reset_counts()
ipns.reset_counts()

# Pick a random node to search for
node_num = random.randint(0, NODE_NUM - 1)
target_content = f"Node {node_num}"
print(f"Looking for node with content: {target_content}")

# Traverse back through the linked nodes to find the target
latest_node_cid = ipns.get_cid(URL)
node = ipfs.retrieve(latest_node_cid)
linked_cids = node.get_linked_cids()
for linked_cid in linked_cids:
    node = ipfs.retrieve(linked_cid)
    if node.get_content() == target_content:
        break

# Output the found node
print(f"Found node: {node}")
get_op_counts()

Looking for node with content: Node 4
Found node: {'CID': 'Qm87ef35440b0bd17ecfd7b19c6521846ba7', 'Content': 'Node 4', 'Linked CID(s)': ['Qm23f625438e3192bd00aaf7ee8fd646c58a', 'Qm418f8568172581af39e284dbcb4f7ea3d3', 'Qma9bee995dbb3079fe3b94ba3a70666fa0a', 'Qm7a8ab4f78d82d0077a34d35dead2d01d15', 'Qm87ef35440b0bd17ecfd7b19c6521846ba7', 'Qm2d05e15f387d5418d84857172be122e0ef', 'Qm738de85d5a732e91e46e1d5a46d66b32fc', 'Qm51d0b7d7011f29c0005a9798308e609881', 'Qme56586218b6af5abd55c23f0d3a451872e', 'Qmff4c038d7f442115fb87b0ba8a33db7b9d', 'Qm43e2dcfc6d4d1ba31c36b04ffb317e69f0', 'Qm6ebbead73d2efa38bc664f759b74168d28', 'Qme2561b91b2bd1233c822dfc512a1adfcb8', 'Qmc2f3346eddc224fb4295416359ced928ae', 'Qm81667eeba364b4319da5e83582ee1e7d18', 'Qm372560932e27de41f8ddb54e01f2676d9e', 'Qm780dc5dca7f26cca14f64dd1f9cda46a02', 'Qme9192edcc76572170ab640f59111fb6d89', 'Qm5481b12732116ba5493c623d3861eddd88', 'Qm97bb6d17366bc717b2c291b7c6084f2d30', 'Qmfca902140b1559803c8b1fecde7c98f226', 'Qm02ca42ac3f0a2748971a

### 3. Linking to previous and first node

In this test, each node will link to the previous node and the first node in the chain.

#### Storing Nodes

In [12]:
# Resetting IPFS from the last test
ipfs.reset_data()

# Testing parameters
NODE_NUM = 100
URL = "example.com"

# Create and store the first node
content = "Node 0"
timestamp = time.time()
to_be_hashed = str({
    "content": content,
    "timestamp": timestamp
})
cid = ipfs.hash(to_be_hashed)
first_node = IPARO(cid=cid, linked_cids=[],
                   content=content, timestamp=timestamp)
ipfs.store(cid, first_node)
ipns.update(URL, cid)

# Create and store the second node
content = "Node 1"
timestamp = time.time()
to_be_hashed = str({
    "content": content,
    "timestamp": timestamp
})
cid = ipfs.hash(to_be_hashed)
linked_cids = [ipns.get_cid(URL)]
second_node = IPARO(cid=cid, linked_cids=linked_cids,
                    content=content, timestamp=timestamp)
ipfs.store(cid, second_node)
ipns.update(URL, cid)

In [13]:
# To automate creating and adding the remaining nodes
for i in range(2, NODE_NUM):
    content = "Node "+str(i)
    timestamp = time.time()
    to_be_hashed = str({
        "content": content,
        "timestamp": timestamp
    })
    cid = ipfs.hash(to_be_hashed)
    latest_node_cid = ipns.get_cid(URL)
    latest_node = ipfs.retrieve(latest_node_cid)
    linked_cids = []
    latest_node_linked_cids = latest_node.get_linked_cids()
    linked_cids.append(latest_node_linked_cids[0])
    linked_cids.append(latest_node_cid)
    node = IPARO(cid=cid, linked_cids=linked_cids,
                 content=content, timestamp=timestamp)
    ipfs.store(cid, node)
    ipns.update(URL, cid)

# print(ipfs.get_data())
get_op_counts()  # Output operation counts

Number of operations IPNS performed:
{'get': 100, 'update': 100}
Number of operations IPFS performed:
{'hash': 100, 'store': 100, 'retrieve': 104}


#### Retrieving Nodes

In [14]:
# Reset the operation counts
ipfs.reset_counts()
ipns.reset_counts()

# Pick a random node to search for
node_num = random.randint(0, NODE_NUM - 1)
target_content = f"Node {node_num}"
print(f"Looking for node with content: {target_content}")

# Check if the first node is the desired node then search the other nodes
latest_node_cid = ipns.get_cid(URL)
node = ipfs.retrieve(latest_node_cid)
linked_cids = node.get_linked_cids()
first_node_cid = linked_cids[0]
first_node = ipfs.retrieve(first_node_cid)
if first_node.get_content() == target_content:
    print(f"Found node: {node}")
else:
    while True:
        node = ipfs.retrieve(linked_cids[1])
        linked_cids = node.get_linked_cids()
        if node.get_content() == target_content:
            break

# Output the found node
print(f"Found node: {node}")
get_op_counts()

Looking for node with content: Node 29
Found node: {'CID': 'Qm6a2146c1fcd5f4c96893be129310b14409', 'Content': 'Node 29', 'Linked CID(s)': ['Qmdd5a49b56e2be2fd701ba9fd501fe0f5cb', 'Qm6d90864f086866bed3fde79a0c275e0262'], 'Timestamp': 1736881215.05603}
Number of operations IPNS performed:
{'get': 1, 'update': 0}
Number of operations IPFS performed:
{'hash': 0, 'store': 0, 'retrieve': 72}


### 4. Linking to K-previous and first node

In this test, each node will link to K previous node and the first node in the chain.

#### Storing Nodes

In [15]:
# Resetting IPFS from the last test
ipfs.reset_data()

# Testing parameters
NODE_NUM = 100
URL = "example.com"
K = 5

# Create and store the first node
content = "Node 0"
timestamp = time.time()
to_be_hashed = str({
    "content": content,
    "timestamp": timestamp
})
cid = ipfs.hash(to_be_hashed)
first_node = IPARO(cid=cid, linked_cids=[],
                   content=content, timestamp=timestamp)
ipfs.store(cid, first_node)
ipns.update(URL, cid)

# Create and store the second node
content = "Node 1"
timestamp = time.time()
to_be_hashed = str({
    "content": content,
    "timestamp": timestamp
})
cid = ipfs.hash(to_be_hashed)
linked_cids = [ipns.get_cid(URL)]
second_node = IPARO(cid=cid, linked_cids=linked_cids,
                    content=content, timestamp=timestamp)
ipfs.store(cid, second_node)
ipns.update(URL, cid)

In [16]:
# To automate creating and adding the remaining nodes
for i in range(2, NODE_NUM):
    content = "Node "+str(i)
    timestamp = time.time()
    to_be_hashed = str({
        "content": content,
        "timestamp": timestamp
    })
    cid = ipfs.hash(to_be_hashed)
    latest_node_cid = ipns.get_cid(URL)
    latest_node = ipfs.retrieve(latest_node_cid)
    latest_node_linked_cids = latest_node.get_linked_cids()
    linked_cids = []
    linked_cids.append(latest_node_linked_cids[0])
    if len(latest_node_linked_cids) == K+1:
        linked_cids.extend(latest_node_linked_cids[2:])
    else:
        linked_cids.extend(latest_node_linked_cids[1:])
    linked_cids.append(latest_node_cid)
    node = IPARO(cid=cid, linked_cids=linked_cids,
                 content=content, timestamp=timestamp)
    ipfs.store(cid, node)
    ipns.update(URL, cid)

get_op_counts()  # Output operation counts

Number of operations IPNS performed:
{'get': 100, 'update': 100}
Number of operations IPFS performed:
{'hash': 100, 'store': 100, 'retrieve': 170}


#### Retrieving Nodes

In [17]:
# Reset the operation counts
ipfs.reset_counts()
ipns.reset_counts()

# Pick a random node to search for
node_num = random.randint(0, NODE_NUM - 1)
target_content = f"Node {node_num}"
print(f"Looking for node with content: {target_content}")

# Check if the first node is the desired node then search the other nodes
latest_node_cid = ipns.get_cid(URL)
node = ipfs.retrieve(latest_node_cid)
linked_cids = node.get_linked_cids()
first_node_cid = linked_cids[0]
first_node = ipfs.retrieve(first_node_cid)
if first_node.get_content() == target_content:
    print(f"Found node: {node}")
else:
    # run through the rest of the linked cids list to check
    for cid in linked_cids:
        node = ipfs.retrieve(cid)
        if node.get_content() == target_content:
            break
    # Get the linked cids of the node at index 1
    node = ipfs.retrieve(linked_cids[1])
    linked_cids = node.get_linked_cids()
    while True:
        # Flag to determine if we should exit the while loop
        found = False

        # Run through the rest of the linked CIDs list to check
        for cid in linked_cids[1:]:
            node = ipfs.retrieve(cid)
            if node.get_content() == target_content:
                found = True
                break

        if found:
            break  # Exit the while loop if the node was found

        if len(linked_cids) >= 2:
            node = ipfs.retrieve(linked_cids[1])
            linked_cids = node.get_linked_cids()
        else:
            print('Can\'t find node')
            break

# Output the found node
print(f"Found node: {node}")
get_op_counts()

Looking for node with content: Node 91
Found node: {'CID': 'Qm9355a204840bdc285e4945c3c8faa22e27', 'Content': 'Node 91', 'Linked CID(s)': ['Qm6a202987ba1cb034e61e0e4abb8b3bb6e4', 'Qmabc58de819e3cb116aa1fb5dbda1d178af', 'Qmc87633e05bdf7a135019cd354f44bd2a91', 'Qm9896962c954ce206262f971710fa7643c1', 'Qmd5c768102f75e2994f5491583ab2e1aa2d', 'Qmfaa576b89b23197f7c09e7be197febc86e'], 'Timestamp': 1736881215.090716}
Number of operations IPNS performed:
{'get': 1, 'update': 0}
Number of operations IPFS performed:
{'hash': 0, 'store': 0, 'retrieve': 12}


### 5. Linking to K-random and first node

In this test, each node will link to a random K previous node and the first node in the chain.

#### Storing Nodes

In [18]:
import random

# Resetting IPFS from the last test
ipfs.reset_data()

# Testing parameters
NODE_NUM = 30
URL = "example.com"
Kmin = 5
Kmax = 10

# Create and store the first node
content = "Node 0"
timestamp = time.time()
to_be_hashed = str({
    "content": content,
    "timestamp": timestamp
})
cid = ipfs.hash(to_be_hashed)
first_node = IPARO(cid=cid, linked_cids=[],
                   content=content, timestamp=timestamp)
ipfs.store(cid, first_node)
ipns.update(URL, cid)

# Create and store the second node
content = "Node 1"
timestamp = time.time()
to_be_hashed = str({
    "content": content,
    "timestamp": timestamp
})
cid = ipfs.hash(to_be_hashed)
linked_cids = [ipns.get_cid(URL)]
second_node = IPARO(cid=cid, linked_cids=linked_cids,
                    content=content, timestamp=timestamp)
ipfs.store(cid, second_node)
ipns.update(URL, cid)

# To automate creating and adding the remaining nodes
for i in range(2, NODE_NUM):
    content = "Node "+str(i)
    timestamp = time.time()
    to_be_hashed = str({
        "content": content,
        "timestamp": timestamp
    })
    cid = ipfs.hash(to_be_hashed)
    latest_node_cid = ipns.get_cid(URL)
    latest_node = ipfs.retrieve(latest_node_cid)
    latest_node_linked_cids = latest_node.get_linked_cids()
    linked_cids = []
    linked_cids.append(latest_node_linked_cids[0])
    K = random.randint(Kmin, Kmax)
    # Check if the number of linked CIDs is greater than K and add K-1 random linked CIDs
    if len(latest_node_linked_cids) > K:
        linked_cids.extend(random.sample(latest_node_linked_cids[1:], K-1))
    # If the number of linked CIDs is less than K add all the linked CIDs
    else:
        linked_cids.extend(latest_node_linked_cids[1:])
    linked_cids.append(latest_node_cid)
    node = IPARO(cid=cid, linked_cids=linked_cids,
                 content=content, timestamp=timestamp)
    print("K = ", K+1)
    print("Length of linked_cids: ", len(linked_cids))
    print(node)
    ipfs.store(cid, node)
    ipns.update(URL, cid)

get_op_counts()  # Output operation counts

K =  11
Length of linked_cids:  2
{'CID': 'Qmd9c89fb03542648cf7b1d141d4e22e0b36', 'Content': 'Node 2', 'Linked CID(s)': ['Qm42a12e2467ce5511d1c96e03c2c303c0f6', 'Qm86087d78d0b855d0a92ba2bf781f8c493b'], 'Timestamp': 1736881215.1131678}
K =  9
Length of linked_cids:  3
{'CID': 'Qma131abddc62f87b8705f454d70ad3f9d6a', 'Content': 'Node 3', 'Linked CID(s)': ['Qm42a12e2467ce5511d1c96e03c2c303c0f6', 'Qm86087d78d0b855d0a92ba2bf781f8c493b', 'Qmd9c89fb03542648cf7b1d141d4e22e0b36'], 'Timestamp': 1736881215.1131678}
K =  9
Length of linked_cids:  4
{'CID': 'Qm45d664af579b789b9e588adfeaba4c2fc1', 'Content': 'Node 4', 'Linked CID(s)': ['Qm42a12e2467ce5511d1c96e03c2c303c0f6', 'Qm86087d78d0b855d0a92ba2bf781f8c493b', 'Qmd9c89fb03542648cf7b1d141d4e22e0b36', 'Qma131abddc62f87b8705f454d70ad3f9d6a'], 'Timestamp': 1736881215.1131678}
K =  10
Length of linked_cids:  5
{'CID': 'Qm8b1ea341dd3ff721af0e6513a1063c4a2f', 'Content': 'Node 5', 'Linked CID(s)': ['Qm42a12e2467ce5511d1c96e03c2c303c0f6', 'Qm86087d78d0b85

#### Retrieving nodes

In [19]:
# Reset the operation counts
ipfs.reset_counts()
ipns.reset_counts()

# Pick a random node to search for
node_num = random.randint(0, NODE_NUM - 1)
target_content = f"Node {node_num}"
print(f"Looking for node with content: {target_content}")

# Check if the first node is the desired node then search the other nodes
latest_node_cid = ipns.get_cid(URL)
node = ipfs.retrieve(latest_node_cid)
linked_cids = node.get_linked_cids()
first_node_cid = linked_cids[0]
first_node = ipfs.retrieve(first_node_cid)
if first_node.get_content() == target_content:
    print(f"Found node: {node}")
else:
    # run through the rest of the linked cids list to check
    for cid in linked_cids:
        node = ipfs.retrieve(cid)
        if node.get_content() == target_content:
            break
    # Get the linked cids of the node at index 1
    node = ipfs.retrieve(linked_cids[1])
    linked_cids = node.get_linked_cids()
    while True:
        # Flag to determine if we should exit the while loop
        found = False

        # Run through the rest of the linked CIDs list to check
        for cid in linked_cids[1:]:
            node = ipfs.retrieve(cid)
            if node.get_content() == target_content:
                found = True
                break

        if found:
            break  # Exit the while loop if the node was found

        if len(linked_cids) >= 2:
            node = ipfs.retrieve(linked_cids[1])
            linked_cids = node.get_linked_cids()
        else:
            print('Can\'t find node')
            break

# Output the found node
print(f"Found node: {node}")
get_op_counts()

Looking for node with content: Node 13
Found node: {'CID': 'Qmedf0dae892a0d93a08e7819adb3a7eeb41', 'Content': 'Node 13', 'Linked CID(s)': ['Qm42a12e2467ce5511d1c96e03c2c303c0f6', 'Qmd9c89fb03542648cf7b1d141d4e22e0b36', 'Qm0f1e2ea8db654f39a953ac129ab2189e2b', 'Qme740fa00ca9def9c503a2b7493eaa63749', 'Qm45d664af579b789b9e588adfeaba4c2fc1', 'Qm49e0a885f332e8dd75dc8c8e20974da5ed', 'Qm0ec41d403cce18ca90a310572a0313ee4f', 'Qma89ed251a33ee0db92d43393451733c1c9'], 'Timestamp': 1736881215.1141665}
Number of operations IPNS performed:
{'get': 1, 'update': 0}
Number of operations IPFS performed:
{'hash': 0, 'store': 0, 'retrieve': 32}


### Other strategies to be tested